In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
file=open("reddit_depression.txt", 'r', encoding="utf8") #open for reading with utf8 encoding
f = file.readlines() #read line by line
#f[:25]
f = [x for x in f if x!='\n'] #interesting code

#delete pattern '%% number\n'
pattern_to_delete_1 = re.compile(r'%% \d\n')

#delete pattern '%% (10 numbers)\n'
pattern_to_delete_2 = re.compile(r'%% \d{10}\n')

for row in range(len(f)):
    f[row] = re.sub(pattern_to_delete_1, '', f[row])
    f[row] = re.sub(pattern_to_delete_2, '', f[row])
    f[row] = f[row].replace('\n','')

f = [x for x in f if x!='']

In [3]:
#f[0:50]

In [4]:
text_data = pd.DataFrame(f)
text_data.columns = ['text']
text_data.shape

#remove all occurences of '\n' sentences as they using extra memmory.
#text_data = text_data[text_data != '\n']

# Drop rows where all data is the same
text_data = text_data.drop_duplicates()

#delete all NaN
#text_data = text_data.dropna()
text_data.shape

(2936749, 1)

In [5]:
#text_data.head(50)

In [6]:
#test = text_data[text_data['text'].str.match(r'(^%% [a-z0-9]{6,9}$)|(^[^%]{2}.*$)')]

In [7]:
test = text_data[text_data['text'].str.match(r'(^%% 7)|(^%% d)|(^[^%].*$)')]
#len(test) - 1546145

In [8]:
#len(test)

In [24]:
new_df = test[:1400] #takes 16 minutes

In [25]:
concat_index = 0
for index, row in new_df.iterrows():
    if ('%% ' in row.text):
        concat_index = 0
    else:
        if concat_index == 0:
            concat_index = index
        else:
            new_df.at[concat_index, 'text'] += row.text
            new_df = new_df.drop(index)
new_df.index = range(len(new_df)) #brings indexes to normal 0:N

In [11]:
#join method ? https://waymoot.org/home/python_string/

In [26]:
#delete IDs without text %% ds0rpzr ; %% ds0vciy ; %% ds0hwwj ; %% ds0ti37 ; %% ds0bj6h
temp_list = new_df["text"].values
to_delete = []

for row in range(0, len(temp_list)):
    if ('%% ' in temp_list[row]):
        if row == 0:
            continue
        else:
            if ('%% ' in temp_list[row-1]):
                to_delete.append(row-1)

modDfObj = new_df.drop(to_delete) #inplace=True - delete from previous dataset

#len(modDfObj[modDfObj['text'].str.match(r'^[^%]')])
#len(modDfObj[modDfObj['text'].str.match(r'%% 7nb')])

In [27]:
len(modDfObj)

92626

In [28]:
ids = []
texts = []

In [29]:
text = modDfObj["text"].values
for idx in range(len(text)):
    if idx % 2 == 0:
        ids.append(text[idx])
    else:
        texts.append(text[idx])

In [30]:
unlab_dataset = pd.DataFrame()
unlab_dataset["id"] = ids
unlab_dataset["text"] = texts

In [31]:
unlab_dataset.head()
#len(unlab_dataset)

,id,text
0,%% 7nbwy8,I've been on and off depression for my entire ...
1,%% ds0p00k,I think that's the point of depression. It's a...
2,%% 7nbwwv,Just a thought I have sometimes. I'm not getti...
3,%% ds0n6pg,It'll probably take a few days for me since I ...
4,%% ds0ncw1,Blue mooners and fair weather friends can go f...


In [78]:
len(unlab_dataset[unlab_dataset["id"].str.match(r'%% 7')])

10723

In [79]:
final = unlab_dataset[unlab_dataset["id"].str.match(r'%% 7')]
#final.index = range(len(final)) #brings indexes to normal 0:N

In [80]:
final.insert(0,'FakeID',range(len(final)))

In [81]:
final.head()

,FakeID,id,text
0,0,%% 7nbwy8,I've been on and off depression for my entire ...
2,1,%% 7nbwwv,Just a thought I have sometimes. I'm not getti...
5,2,%% 7nbw09,I loved you at the beginning 2017. You took me...
7,3,%% 7nbvyp,I just want to be myself again.I am full of an...
8,4,%% 7nbvo4,"Every ""day"" is the fucking same. Wake up, show..."


In [82]:
len(final)

10723

In [83]:
# you can also write data to a file
final.to_csv("final_orig.csv", sep='\t', index=False)

In [84]:
final.to_csv("map.csv", sep='\t', columns = ['FakeID','id'], index=False)

In [85]:
final.to_csv("final_fake.csv", sep='\t',columns = ['FakeID', 'text'], index=False)

In [86]:
data = pd.read_csv("final_fake.csv", sep='\t')

In [87]:
#del data['Unnamed: 0']

In [88]:
data.head()

,FakeID,text
0,0,I've been on and off depression for my entire ...
1,1,Just a thought I have sometimes. I'm not getti...
2,2,I loved you at the beginning 2017. You took me...
3,3,I just want to be myself again.I am full of an...
4,4,"Every ""day"" is the fucking same. Wake up, show..."
